In [5]:
"""
1. 첫 번째 소인수 분해 알고리즘
입력 : 1보다 큰 정수 n
출력 : 소인수와 차수(order)의 딕셔너리
- {p1:k1, p2:k2, ... }
2부터 sqrt(n)까지 정수 i에 대해 순차적으로
- i가 n의 약수라면 딕셔너리에 추가 (차수는 1로 초기화)
- 딕셔너리에 이미 해당 약수가 존재하면 차수를 1 증가
- n은 n을 i로 나눈 몫으로 변경
- i가 n의 약수가 아닐 때까지 반복
"""

import math

In [6]:
# 소인수 분해
def factorizel(n):
    factors = {}
    sqrtn = math.floor(math.sqrt(n))
    for i in range(2, sqrtn + 1):
        while (n % i == 0):
            addFactor(factors, i)
            n = n // i
    if (n > 1):
        addFactor(factors, n)
    return factors

# 약수 추가
def addFactor(factors, f):
    if (f not in factors):
        factors[f] = 1 # 소인수 추가
    else:
        factors[f] += 1 # 차수 1 증가

In [22]:
import time
start = time.time()

n = (2**10) - 1
print('2^10 - 1:', factorizel(n))
    
end = time.time()
print("실행시간:", end-start)

2^10 - 1: {3: 1, 11: 1, 31: 1}
실행시간: 0.0


In [9]:
"""
성능 개선을 위한 아이디어
- 소인수는 모두 소수이므로 1부터 sqrt(n)까지의 소수를 찾으면 됨
-> primes
primes에 있는 소수들로만 소인수 분해하면 됨
"""

def findPrimes(n):
    primes = []
    for i in range(2, n+1):
        if (isPrime(i)):
            primes.append(i)
    return primes

# 소수 판별
def isPrime(n):
    sqrtn = math.floor(math.sqrt(n))
    for i in range(2, sqrtn + 1):
        if (n % i == 0):
            return False
    return True

"""
모듈러 연산 - 나눗셈 연산 - 뺄셈 연산 - 2의 보수 덧셈 연산 - 비트 연산
알고리즘 단위 연산에 영향을 미치는 입력 크기는 n의 비트 수
n의 비트수 b = |log2(n)|+1,
시간 복잡도 O(sqrt(n))=O(sqrt(2^b))=O(2^(b/2)) -> 지수복잡도

에라토스테네스의 체, 확장된 에라토스테네스의 체로 조금 더 효율적일 수 있음
"""

start = time.time()
n = (2**21) - 1
print(n, ":", len(findPrimes(n)))
end = time.time()
print("실행시간:", end-start)

2097151 : 155611
실행시간: 9.634851694107056


In [14]:
"""
에라토스테네스의 체
2<=i<=sprt(n) 에서 작은 순대로 소수 판별
a_i가 소수이면 k*a_i는 합성수
-> k*a_i만 제외하면 소수를 얻음
"""

# 체
def sieve(n):
    flags = [0, 0] + [1] * (n-1) # 0, 1, 2, ... n
    sqrtn = math.floor(math.sqrt(n))
    for i in range(2, sqrtn + 1):
        if (flags[i] == 1):
            for j in range(2*i, n+1, i): #2 2i부터 n까지 i씩 더해줌
                flags[j] = 0
    primes = []
    for i in range(len(flags)):
        if (flags[i] == 1):
            primes.append(i)
    return primes

start = time.time()
n = (2**21) - 1
print(n, ":", len(sieve(n)))
end = time.time()
print("실행시간:", end-start)

2097151 : 155611
실행시간: 0.33449649810791016


In [21]:
"""
에라토스테네스의 체의 문제점
1. 메모리는 충분한가
- flags 배열을 저장할 수 있는 충분한 메모리 공간
- 그래도 n이 너무 크다면?
-> 세그먼트 에라토스테네스의 체
    - n이 충분히 크면, 1~n을 sqrt(n)개의 세그먼트로 분할
    - 첫번째 세그먼트(1~sqrt(n))에서 소수를 모두 찾은 뒤
    - 이 소수들로 다음 세그먼트들을 차례로 방문하여 처리
    
첫 번째 세그먼트에서 필요한 소수들을 seeds에 담아 둠
이 소수들로 나머지 모든 세그먼트에서 배수를 지움
각 세그먼트에서 배수를 지우고 남은 소수들을 primes에 담음
seeds + primes가 최종적인 소수 집합임
"""

def segemented(n):
    sqrtn = math.floor(math.sqrt(n))
    seeds = sieve(sqrtn)
    primes = []
    low = sqrtn # 세그먼트의 시작
    high = 2*sqrtn - 1 # 세그먼트의 마지막
    
    while (low <= n):
        if (high > n): # 마지막 세그먼트일 때
            high = n 
        
        flags = [1]*sqrtn
        for i in range(len(seeds)):
            
            # 시작 위치
            start = (low // seeds[i]) * seeds[i]
            if(start < low): start += seeds[i]
            
            # i의 배수 제거
            for j in range(start, high + 1, seeds[i]):
                flags[j - low] = 0 # flag는 0부터 시작이므로
        for i in range(low, high +1):
            if (flag[i - low] == 1):
                primes.append(i)
        
        low += sqrtn
        high += sqrtn
    return seeds + primes

"""
에라토스테네스의 체의 시간 복잡도
알고리즘의 시간 복잡도가 O(nlog(logn))임이 증명ehla
log(logn) ~ 1, O(nlog(logn)) ~ O(n) -> near linear
그러나 n의 비트 수 b에 대해서는 여전히 O(2^b), 지수시간
"""

start = time.time()
n = (2**21) - 1
print(n, ":", len(sieve(n)))
end = time.time()
print("실행시간:", end-start)
# 공간복잡도가 효율적인 반면 세그먼트들을 옮기는 과정 때문에 시간 복잡도는 올라감

2097151 : 155611
실행시간: 0.3195343017578125
